In [1]:
import torch
import csv
import os
import torchvision.transforms.v2 as transforms_v2
from torch import Tensor
from typing import Callable, List, Optional, Type, Union
from PIL import Image
from torch.utils.data import DataLoader
from torch.utils.data import TensorDataset
from torch import nn
device = "cuda" if torch.cuda.is_available() else "cpu"
torch.cuda.set_device(1)

In [ ]:
train_images = [] 
train_labels = [] 
class_to_idx = {}

transform_before = transforms_v2.Compose([
                            transforms_v2.ToImage(),
                            transforms_v2.ToDtype(dtype = torch.float32, scale=True),
                            transforms_v2.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
                        ])

transform_train = transforms_v2.Compose([
                            transforms_v2.RandomResizedCrop(64, scale=(0.6, 1.0), ratio=(0.8, 1.2)),
                            transforms_v2.RandomHorizontalFlip(p=0.5),
                            transforms_v2.ColorJitter(
                                brightness=0.4,
                                contrast=0.4,
                                saturation=0.4,
                                hue=0.1
                            ),
                            transforms_v2.RandomAffine(
                                degrees=15,
                                translate=(0.1, 0.1),
                            ),
                            transforms_v2.RandomGrayscale(p=0.1)
                        ])

class_folders = os.listdir(r"C:\Users\sjw00\OneDrive\Desktop\dataset\tiny-imagenet-200\train")
for num, i in enumerate(class_folders):
    class_to_idx[i] = num

for class_folder in class_folders:
    folder_path = os.path.join(r"C:\Users\sjw00\OneDrive\Desktop\dataset\tiny-imagenet-200\train", class_folder) +r"\images"
    temp_image = []
    temp_label = []
    for img_file in os.listdir(folder_path): 
        img_path = os.path.join(folder_path, img_file) 

        img = Image.open(img_path)
        if img.mode != 'RGB':
            img = img.convert('RGB')
        img_tensor = transform_before(img)  

        train_images.append(img_tensor)
        train_labels.append(torch.tensor(class_to_idx[class_folder],dtype=torch.long))

train_images = torch.stack(train_images)
train_labels = torch.stack(train_labels)
########################################################################
test_images = [] 
test_labels = [] 

with open(r"C:\Users\sjw00\OneDrive\Desktop\dataset\tiny-imagenet-200\val\val_annotations.txt", 'r') as f:
    for line in f:
        file, idx, _ = line.strip().split('\t', 2) # Split each line by space into folder name and index
        img_path = os.path.join(r"C:\Users\sjw00\OneDrive\Desktop\dataset\tiny-imagenet-200\val\images", file) 
        
        img = Image.open(img_path)
        if img.mode != 'RGB':
            img = img.convert('RGB')
        img_tensor = transform_before(img)  

        test_images.append(img_tensor)
        test_labels.append(torch.tensor(class_to_idx[idx],dtype=torch.long))

test_images = torch.stack(test_images).to("cuda")
test_labels = torch.stack(test_labels).to("cuda")

training_dataset = TensorDataset(train_images, train_labels)
test_dataset = TensorDataset(test_images, test_labels)

training_dataloader = DataLoader(training_dataset, batch_size=500, shuffle=True)
test_dataloader = DataLoader(test_dataset, batch_size=500, shuffle=False)


In [ ]:
def conv3x3(in_planes: int, out_planes: int, stride: int = 1, groups: int = 1, dilation: int = 1) -> nn.Conv2d:
    """3x3 convolution with padding"""
    return nn.Conv2d(
        in_planes,
        out_planes,
        kernel_size=3,
        stride=stride,
        padding=dilation,
        groups=groups,
        bias=False,
        dilation=dilation,
    )

def conv1x1(in_planes: int, out_planes: int, stride: int = 1) -> nn.Conv2d:
    """1x1 convolution"""
    return nn.Conv2d(in_planes, out_planes, kernel_size=1, stride=stride, bias=False)

class Bottleneck(nn.Module):
    # Bottleneck in torchvision places the stride for downsampling at 3x3 convolution(self.conv2)
    # while original implementation places the stride at the first 1x1 convolution(self.conv1)
    # according to "Deep residual learning for image recognition" https://arxiv.org/abs/1512.03385.
    # This variant is also known as ResNet V1.5 and improves accuracy according to
    # https://ngc.nvidia.com/catalog/model-scripts/nvidia:resnet_50_v1_5_for_pytorch.

    expansion: int = 4

    def __init__(
        self,
        inplanes: int,
        planes: int,
        stride: int = 1,
        downsample: Optional[nn.Module] = None,
        groups: int = 1,
        base_width: int = 64,
        dilation: int = 1,
        norm_layer: Optional[Callable[..., nn.Module]] = None,
    ) -> None:
        super().__init__()
        if norm_layer is None:
            norm_layer = nn.BatchNorm2d
        width = int(planes * (base_width / 64.0)) * groups
        # Both self.conv2 and self.downsample layers downsample the input when stride != 1
        self.conv1 = conv1x1(inplanes, width)
        self.bn1 = norm_layer(width)
        self.conv2 = conv3x3(width, width, stride, groups, dilation)
        self.bn2 = norm_layer(width)
        self.conv3 = conv1x1(width, planes * self.expansion)
        self.bn3 = norm_layer(planes * self.expansion)
        self.relu = nn.ReLU()
        self.downsample = downsample
        self.stride = stride
    def forward(self, x: Tensor) -> Tensor:
        identity = x

        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)

        out = self.conv2(out)
        out = self.bn2(out)
        out = self.relu(out)

        out = self.conv3(out)
        out = self.bn3(out)

        if self.downsample is not None:
            identity = self.downsample(x)
        #print("ㅎ")
        out += identity
        out = self.relu(out)

        return out
    
class ResNet(nn.Module):
    def __init__(
        self,
        block: Type[Bottleneck],
        layers: List[int],
        num_classes: int = 1000,
        zero_init_residual: bool = False,
        groups: int = 1,
        width_per_group: int = 64,
        replace_stride_with_dilation: Optional[List[bool]] = None,
        norm_layer: Optional[Callable[..., nn.Module]] = None,
    ) -> None:
        super().__init__()
      
        if norm_layer is None:
            norm_layer = nn.BatchNorm2d
        self._norm_layer = norm_layer

        self.inplanes = 64
        self.dilation = 1
        if replace_stride_with_dilation is None:
            # each element in the tuple indicates if we should replace
            # the 2x2 stride with a dilated convolution instead
            replace_stride_with_dilation = [False, False, False]
        if len(replace_stride_with_dilation) != 3:
            raise ValueError(
                "replace_stride_with_dilation should be None "
                f"or a 3-element tuple, got {replace_stride_with_dilation}"
            )
        self.groups = groups
        self.base_width = width_per_group
        self.conv1 = nn.Conv2d(3, self.inplanes, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn1 = norm_layer(self.inplanes)
        self.relu = nn.ReLU()
        self.maxpool = nn.MaxPool2d(kernel_size=3, stride=2, padding=1)
        self.layer1 = self._make_layer(block, 64, layers[0])
        self.layer2 = self._make_layer(block, 128, layers[1], stride=2, dilate=replace_stride_with_dilation[0])
        self.layer3 = self._make_layer(block, 256, layers[2], stride=2, dilate=replace_stride_with_dilation[1])
        self.layer4 = self._make_layer(block, 512, layers[3], stride=2, dilate=replace_stride_with_dilation[2])
        self.avgpool = nn.AdaptiveAvgPool2d((1, 1))
        self.fc = nn.Linear(512 * block.expansion, num_classes)
        #torch.nn.init.zeros_(self.fc.weight)
        """
        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                nn.init.kaiming_normal_(m.weight, mode="fan_out", nonlinearity="relu")
            elif isinstance(m, (nn.BatchNorm2d, nn.GroupNorm)):
                nn.init.constant_(m.weight, 1)
                nn.init.constant_(m.bias, 0)
        """
        # Zero-initialize the last BN in each residual branch,
        # so that the residual branch starts with zeros, and each residual block behaves like an identity.
        # This improves the model by 0.2~0.3% according to https://arxiv.org/abs/1706.02677

    def _make_layer(
        self,
        block: Type[Union[Bottleneck]],
        planes: int,
        blocks: int,
        stride: int = 1,
        dilate: bool = False,
    ) -> nn.Sequential:
        norm_layer = self._norm_layer
        downsample = None
        previous_dilation = self.dilation
        if dilate:
            self.dilation *= stride
            stride = 1
        if stride != 1 or self.inplanes != planes * block.expansion:
            downsample = nn.Sequential(
                conv1x1(self.inplanes, planes * block.expansion, stride),
                norm_layer(planes * block.expansion),
            )

        layers = []
        layers.append(
            block(
                self.inplanes, planes, stride, downsample, self.groups, self.base_width, previous_dilation, norm_layer
            )
        )
        self.inplanes = planes * block.expansion
        for _ in range(1, blocks):
            layers.append(
                block(
                    self.inplanes,
                    planes,
                    groups=self.groups,
                    base_width=self.base_width,
                    dilation=self.dilation,
                    norm_layer=norm_layer,
                )
            )

        return nn.Sequential(*layers)

    def _forward_impl(self, x: Tensor) -> Tensor:
        # See note [TorchScript super()]
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu(x)
        x = self.maxpool(x)
        #print(x.shape)
        x = self.layer1(x)
        #print(x.shape)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)
        #print(x.shape)
        x = self.avgpool(x)
        x = torch.flatten(x, 1)
        x = self.fc(x)

        return x

    def forward(self, x: Tensor) -> Tensor:
        return self._forward_impl(x)

In [ ]:
def train(dataloader, model, loss_fn, optimizer, scaler, transform_train):   
    for batch, (X, y) in enumerate(dataloader):
        X = transform_train(X).to("cuda")
        y = y.to("cuda")
        with torch.autocast(device_type='cuda', dtype=torch.bfloat16):
            pred = model(X)
            batch_loss_result = loss_fn(pred, y)
        optimizer.zero_grad()

        scaler.scale(batch_loss_result).backward() # Backpropagation: compute gradients
        scaler.step(optimizer) # Update model parameters using the optimizer
        scaler.update() # Update the gradient scaler


In [ ]:
def test(dataloader, model):
    with torch.no_grad():
        accuracy_sum=0
        for _, (X, y) in enumerate(dataloader):
            with torch.autocast(device_type='cuda', dtype=torch.bfloat16):
                pred = model(X)
            accuracy_sum+= (torch.argmax(pred, dim=1) == y).type(torch.float).sum().item()

        print("test_accuracy", accuracy_sum/10000)
    return accuracy_sum/10000

In [ ]:
summary = list()
total_list = []
for i in range(1):
    model = ResNet(Bottleneck,[3,4,6,3],num_classes=200).to("cuda")
    loss_fn = nn.CrossEntropyLoss()
    scaler = torch.amp.GradScaler()
    optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)
    scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=20, gamma=0.95)
    accuracy = 0 
    epoch_list = []
    for t in range(500):
        print(f"Epoch {t+1}\n-------------------------------")
        train(training_dataloader, model, loss_fn, optimizer, scaler, transform_train)
        accuracy_temp = test(test_dataloader, model)
        epoch_list.append(accuracy_temp)
        scheduler.step()
        if accuracy_temp > accuracy:
            accuracy = accuracy_temp
    total_list.append(epoch_list)
    summary.append(accuracy)
    print(summary)
with open("tiny_1.csv", mode="w", newline="", encoding="utf-8") as file:
    writer = csv.writer(file)
    writer.writerows(total_list)
print("Done!")
print(summary)